1. Collect traces, fit model, sample Weibull posterior):
    * Observation To = traces before the shift
    * Fit Weibull for Observation To $Wo_0$
    * Observation T1 = traces after the shift
    * Fit Weibull for Observation T1 $Wo_1$

In [ ]:
import pytensor
import pandas as pd
from pathlib import Path
import pymc as pm
import arviz as az
from pandas.io.json import to_json
from pymc import predictions_to_inference_data
from scipy.stats import weibull_min

pytensor.config.cxx = ''
pytensor.config.floatX = "float64"

In [ ]:
def fit_observation_distributions(obs_t0_path, obs_t1_path, should_sample_posterior: bool = False,
                                  idata_file_name: str = '', observations: dict[str, pd.DataFrame] = None):
    # obs_t0_path = Path().joinpath('distribution_shift', 'traces', 'scratch_s50_f0.5_shift_s50_f0.5', '.summary.csv')
    # obs_t1_path = Path().joinpath('distribution_shift', 'traces', 'scratch_s50_f0.5_shift_s50_f1.0', '.summary.csv')

    if not observations:
        obs_df = {
            't0': pd.read_csv(obs_t0_path),
            't1': pd.read_csv(obs_t1_path)
        }
    else:
        obs_df = observations

    # Transform to prevent 0 values in log calculation
    for key, df in obs_df.items():
        if 'collisions' in df.columns:
            df['collisions'] = df['collisions'] + 0.01

    priors_weibull = {
        't0': {'collisions': {}, 'waitingTime': {}},
        't1': {'collisions': {}, 'waitingTime': {}}
    }

    obs_weibull = {
        't0': {'collisions': pm.Model(), 'waitingTime': pm.Model()},
        't1': {'collisions': pm.Model(), 'waitingTime': pm.Model()}
    }

    idata_weibull = {
        't0': {'collisions': None, 'waitingTime': None},
        't1': {'collisions': None, 'waitingTime': None}
    }

    for time in priors_weibull.keys():
        for variable in priors_weibull[time].keys():
            print(f'Fitting {variable} at {time}')
            # Replace with your observed data
            data = obs_df[time][variable]

            # Fit Weibull using MLE (shape, loc, scale)
            priors_weibull[time][variable]['shape'], _, priors_weibull[time][variable]['scale'] = weibull_min.fit(data,
                                                                                                                  floc=0)

            print(f"{time} {variable}: Estimated shape (α): {priors_weibull[time][variable]['shape']:.3f}")
            print(f"{time} {variable}: Estimated scale (β): {priors_weibull[time][variable]['scale']:.3f}")

    for time in obs_weibull.keys():
        for variable in obs_weibull[time].keys():
            print(f'Fitting {variable} at {time}')
            with obs_weibull[time][variable]:
                alpha = pm.TruncatedNormal("alpha", mu=priors_weibull[time][variable]['shape'],
                                           sigma=priors_weibull[time][variable]['shape'] * 0.5, lower=0.01)
                beta = pm.TruncatedNormal("beta", mu=priors_weibull[time][variable]['scale'],
                                          sigma=priors_weibull[time][variable]['scale'] * 0.5, lower=0.01)
                obs = pm.Weibull(variable, alpha=alpha, beta=beta, observed=obs_df[time][variable])

                if idata_file_name:
                    idata_path = Path().joinpath('bayesian_models', f'obs_{idata_file_name}_{time}_{variable}.netcdf')
                else:
                    idata_path = Path().joinpath('bayesian_models', f'obs_weibull_{time}_{variable}.netcdf')

                if not idata_path.exists():
                    idata = pm.sample()
                    idata.to_netcdf(idata_path)
                else:
                    idata = az.from_netcdf(idata_path)
                idata_weibull[time][variable] = idata

    predictions = {
        't0': {'collisions': None, 'waitingTime': None},
        't1': {'collisions': None, 'waitingTime': None}
    }

    if should_sample_posterior:
        for time in obs_weibull.keys():
            for variable in obs_weibull[time].keys():
                with obs_weibull[time][variable]:
                    idata = az.InferenceData.from_netcdf(
                        str(Path().joinpath('bayesian_models', f'obs_weibull_{time}_{variable}.netcdf')))
                    predictions[time][variable] = pm.sample_posterior_predictive(idata, predictions=True,
                                                                                 return_inferencedata=False).predictions

    return idata_weibull, predictions

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


def extract_split_dfs(prediction, obs_df, variable):
    data = {}
    for time_key in ['t0', 't1']:
        # Predicted
        pred = pd.DataFrame({
            variable: prediction[time_key][variable][variable].values.flatten(),
        })
        pred['time'] = time_key
        pred['source'] = 'predicted'
        data[f'{time_key}_predicted'] = pred

        # Observed
        obs = pd.DataFrame({
            variable: obs_df[time_key][variable].values
        })
        obs['time'] = time_key
        obs['source'] = 'observed'
        data[f'{time_key}_observed'] = obs
    return data


def plot_comparison(df1, df2, variable, title):
    combined = pd.concat([df1, df2], ignore_index=True)
    plt.figure(figsize=(10, 5))
    sns.histplot(
        data=combined,
        x=variable,
        hue='source',
        stat='density',
        common_norm=False,  # Key fix!
        bins=30,
        element='step',
        fill=True,
        alpha=0.4
    )
    plt.title(title)
    plt.xlabel(variable)
    plt.ylabel('Density')
    plt.tight_layout()
    plt.show()


def plot_variable_distributions(prediction, obs_df, variable):
    data = extract_split_dfs(prediction, obs_df, variable)

    # 1. Predicted vs Observed at same time
    for time_key in ['t0', 't1']:
        plot_comparison(
            data[f'{time_key}_predicted'],
            data[f'{time_key}_observed'],
            variable,
            f'{variable.capitalize()} — {time_key.upper()}: Predicted vs Observed'
        )

    # 2. Observed t0 vs t1
    observed_t0 = data['t0_observed'].copy()
    observed_t1 = data['t1_observed'].copy()
    observed_t0['source'] = 'observed_t0'
    observed_t1['source'] = 'observed_t1'
    plot_comparison(observed_t0, observed_t1, variable, f'{variable.capitalize()} — Observed t0 vs t1')

    # 3. Predicted t0 vs t1
    predicted_t0 = data['t0_predicted'].copy()
    predicted_t1 = data['t1_predicted'].copy()
    predicted_t0['source'] = 'predicted_t0'
    predicted_t1['source'] = 'predicted_t1'
    plot_comparison(predicted_t0, predicted_t1, variable, f'{variable.capitalize()} — Predicted t0 vs t1')


### Causal Discovery

In [ ]:
from castle.common.priori_knowledge import PrioriKnowledge
import networkx as nx
import pandas as pd
import itertools
from castle.algorithms import PC
from pathlib import Path
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

# Configuration constants
DATA_PATH = Path('structural_causal_models', 'data', 'scm_observational_data.csv')
SELECTED_COLUMNS = [
    'desiredSpeed', 'friction', 'speed',
    'waitingTime', 'emergencyBraking', 'collisions'
]
INDEPENDENT_VARIABLES = ['desiredSpeed', 'friction']
OUTCOME_VARIABLES = ['waitingTime', 'collisions']


def create_priori_knowledge(dataframe: pd.DataFrame) -> PrioriKnowledge:
    """Create prior knowledge constraints for causal discovery."""
    pk = PrioriKnowledge(len(dataframe.columns))
    forbidden_edges = []

    # Prevent edges into independent variables
    for independent_variable in INDEPENDENT_VARIABLES:
        forbidden_edges.extend((column_name, independent_variable) for column_name in dataframe.columns if
                               column_name != independent_variable)

    # Prevent edges between outcome variables
    forbidden_edges.extend(itertools.permutations(OUTCOME_VARIABLES, 2))

    # Convert to column indices and add to prior knowledge
    edge_indices = [
        (dataframe.columns.get_loc(source), dataframe.columns.get_loc(target))
        for source, target in forbidden_edges
    ]
    pk.add_forbidden_edges(edge_indices)

    return pk


def compute_edge_direction_confidence(graph: nx.DiGraph, dataframe: pd.DataFrame) -> dict:
    """Compute confidence scores for edge directions using regression residuals."""
    edge_confidence = {}

    for edge in graph.edges():
        x, y = edge
        x_data = dataframe[x].values.reshape(-1, 1)
        y_data = dataframe[y].values.reshape(-1, 1)

        # Model X = f(Y)
        model_x_y = LinearRegression()
        model_x_y.fit(y_data, x_data)
        residuals_x = x_data - model_x_y.predict(y_data)

        # Model Y = g(X)
        model_y_x = LinearRegression()
        model_y_x.fit(x_data, y_data)
        residuals_y = y_data - model_y_x.predict(x_data)

        # Compute Spearman correlations
        corr_x_resid, _ = spearmanr(x_data.ravel(), residuals_y.ravel())
        corr_y_resid, _ = spearmanr(y_data.ravel(), residuals_x.ravel())

        confidence = abs(abs(corr_y_resid) - abs(corr_x_resid))
        edge_confidence[(x, y) if abs(corr_x_resid) < abs(corr_y_resid) else (y, x)] = confidence

    return edge_confidence


def adjust_edges(original_graph: nx.DiGraph, edge_confidence: dict) -> list:
    """Adjust edge directions based on confidence scores and prior knowledge. If confidence is too low (<0.1) originally discovered edge is preserved"""
    adjusted_edges = []

    for edge, confidence in edge_confidence.items():
        if edge not in original_graph.edges():
            if confidence < 0.1 or edge[1] in INDEPENDENT_VARIABLES:
                edge = (edge[1], edge[0])
        adjusted_edges.append(edge)

    return adjusted_edges


def create_adjusted_graph(original_graph: nx.DiGraph, adjusted_edges: list) -> nx.DiGraph:
    """Create a new graph with adjusted edges."""
    new_graph = original_graph.copy()
    new_graph.clear_edges()
    new_graph.add_edges_from(adjusted_edges)
    return new_graph


def visualize_causal_graph(graph: nx.DiGraph, title: str) -> None:
    """Visualize the causal graph with specified coloring scheme."""
    plt.figure(figsize=(10, 6))
    plt.title(title)

    color_map = [
        'green' if node in INDEPENDENT_VARIABLES else
        'red' if graph.out_degree(node) == 0 else
        'yellow'
        for node in graph.nodes
    ]

    nx.draw(G=graph, node_color=color_map, node_size=1200, arrowsize=30, with_labels=True,
            pos=nx.circular_layout(graph))


# Begin
df = pd.read_csv(DATA_PATH)[SELECTED_COLUMNS]

# Causal discovery setup
priori_knowledge = create_priori_knowledge(df)
pc = PC(variant='stable', priori_knowledge=priori_knowledge)
pc.learn(df.values.tolist())

# Graph processing
causal_graph = nx.DiGraph(pc.causal_matrix)
labeled_graph = nx.relabel_nodes(causal_graph, dict(enumerate(df.columns)))

# Edge direction adjustment
edge_confidence = compute_edge_direction_confidence(labeled_graph, df)
adjusted_edges = adjust_edges(labeled_graph, edge_confidence)
adjusted_graph = create_adjusted_graph(labeled_graph, adjusted_edges)

# Get The Reverse
reversed_graph = adjusted_graph.reverse()
reversed_graph.remove_nodes_from(INDEPENDENT_VARIABLES)

# Visualization
visualize_causal_graph(labeled_graph, "Discovered Causal Graph")
visualize_causal_graph(adjusted_graph, "Direction Adjusted Causal Graph")
visualize_causal_graph(reversed_graph, "Reversed Mediator Causal Graph")

Get Bayesian Modle From Graph

In [ ]:
import pytensor
from pathlib import Path
import pandas as pd
import pymc as pm
import arviz as az
import networkx as nx

pytensor.config.cxx = ''
pytensor.config.floatX = "float64"


def generate_scm_layers(structural_causal_graph: nx.DiGraph) -> [dict, dict]:
    assert nx.is_directed_acyclic_graph(structural_causal_graph)
    node_layers = {node: 0 for node in structural_causal_graph.nodes}
    predecessors = {node: list(structural_causal_graph.predecessors(node)) for node in structural_causal_graph.nodes}
    visited = set(node for node, predecessor_nodes in predecessors.items() if len(predecessor_nodes) == 0)

    # Find layers for each node
    current_layer = max(node_layers.values()) + 1
    while len(visited) < len(structural_causal_graph.nodes):
        unprocessed = set(structural_causal_graph.nodes) - visited
        current_nodes = [node for node in unprocessed if visited.issuperset(predecessors[node])]
        for node in current_nodes:
            node_layers[node] = current_layer
        visited.update(current_nodes)
        current_layer += 1
    return node_layers, predecessors


def generate_scm_effect_model(data: pd.DataFrame, structural_causal_graph: nx.DiGraph,
                              effect: str = 'total') -> pm.Model:
    """Generate SCM model for either total or direct effects."""

    if effect not in {'total', 'direct'}:
        raise ValueError("Effect must be either 'total' or 'direct'")

    node_layers, predecessors = generate_scm_layers(structural_causal_graph)
    outcome_variables = [node for node in structural_causal_graph.nodes if
                         structural_causal_graph.out_degree(node) == 0]
    df_standardized = (data - data.mean()) / data.std()
    values = {column: df_standardized[column].values for column in df_standardized.columns}
    alpha, beta, sigma, mu, obs, independent_data = {}, {}, {}, {}, {}, {}

    with pm.Model() as structural_causal_model:
        for node in sorted(node_layers.keys(), key=node_layers.__getitem__):
            node_should_be_observed = (effect == 'total' and predecessors[node]) or (
                    effect == 'direct' and node in outcome_variables)
            if node_should_be_observed:
                alpha[node] = pm.Normal(f"{node} alpha", mu=0, sigma=10)

                current_predecessor_string = ', '.join(
                    f'{index}:{name}' for index, name in enumerate(predecessors[node]))
                current_node_beta_name = f"{node} beta * ({current_predecessor_string})"

                beta[node] = pm.Normal(current_node_beta_name, mu=0, sigma=10, shape=len(predecessors[node]))
                sigma[node] = pm.HalfNormal(f"{node} sigma", sigma=1)

                mu[node] = alpha[node]
                for i, predecessor in enumerate(predecessors[node]):
                    if effect == 'total':
                        predictor = independent_data[predecessor] if node_layers[predecessor] == 0 else obs[predecessor]
                    else:
                        predictor = independent_data[predecessor]
                    mu[node] += beta[node][i] * predictor

                obs[node] = pm.Normal(f"{node} obs", mu=mu[node], sigma=sigma[node], observed=values[node])
            else:
                # Make sure independent variables are modeled
                independent_data[node] = pm.Data(f"{node}_data", values[node])
    return structural_causal_model


def shorten_graph_nodes(graph: nx.DiGraph) -> nx.DiGraph:
    """Renames nodes in a graph copy according to specified abbreviation rules."""

    def abbreviate(name: str) -> str:
        """Abbreviates variable names using first letter and uppercase letters."""
        if not name:
            return ""
        abbreviation = [name[0].lower()]
        for c in name[1:]:
            if c.isupper():
                abbreviation.append(c)
        return ''.join(abbreviation)

    def process_name(name: str) -> str:
        """Processes a single node name through all transformation rules."""
        # Remove parentheses content
        clean_name = name.split('*')[0].strip()
        parts = clean_name.split('_') if '_' in clean_name else clean_name.split()

        if not parts:
            return name  # Return original if empty after cleaning

        # Process variable part
        variable = abbreviate(parts[0])
        components = [variable]

        # Process remaining parts
        i = 1
        while i < len(parts):
            part = parts[i]
            if part == "alpha":
                components.append("α")
            elif part == "beta":
                components.append("β*")
            elif part == "sigma":
                components.append("σ")
            elif part in {"obs", "data"}:
                components.append(part)
            i += 1

        return ' '.join(components)

    # Create mapping and return relabeled graph
    return nx.relabel_nodes(graph, {n: process_name(n) for n in graph.nodes}, copy=True)


def visualize_scm_model(model: pm.Model) -> None:
    model_graph = pm.model_to_networkx(model)
    model_graph = shorten_graph_nodes(model_graph)

    plt.figure(figsize=(20, 20))  # Set viewport size (width, height in inches)

    nx.draw(
        model_graph,
        pos=nx.planar_layout(model_graph, scale=1),  # Increase node spacing
        with_labels=True,
        node_size=2500,  # Increase node size (default 300)
        node_color='skyblue',  # Better visibility
        edge_color='gray',  # Better visibility
        width=3,  # Increase edge thickness (default 1)
        font_size=14,  # Increase label size
        font_weight='bold',  # Improve label readability
        arrowsize=15  # Increase arrow size for directed edges
    )

    plt.show()


scm_total_effect_model = generate_scm_effect_model(df, adjusted_graph, effect='total')
scm_direct_effect_model = generate_scm_effect_model(df, adjusted_graph, effect='direct')
scm_reverse_model = generate_scm_effect_model(df, reversed_graph, effect='total')

visualize_scm_model(scm_total_effect_model)
visualize_scm_model(scm_direct_effect_model)
visualize_scm_model(scm_reverse_model)

In [ ]:
with scm_reverse_model:
    idata = pm.sample()
    idata.to_netcdf(Path().joinpath('structural_causal_models', 'scm_reverse.netcdf'))

az.summary(idata, round_to=2)

3. Def: function estimate_posterior_intervention (data, intervention_range)
    * z-score normalize the Observation To,
    * sample a normal from the fitted SCM,
    * de-normalize the data,
    * fit the Weibull (to guarantee positive values and make all distributions comparable),
    * sample the Weibull back to obtain actual outcome values (collision, waiting time)

In [ ]:
import numpy as np

SCM_SOURCE_DATA = pd.read_csv(Path().joinpath('structural_causal_models', 'data', 'scm_observational_data.csv'))[
    ['desiredSpeed', 'friction', 'speed', 'waitingTime', 'emergencyBraking', 'collisions']]

OUTCOME_VARIABLES = ['waitingTime', 'collisions']
INTERVENTION_VARIABLES = ['desiredSpeed', 'friction']
REVERSED_MODEL_IDATA = Path().joinpath('structural_causal_models', 'scm_reverse.netcdf')


def sample_posterior(model: pm.Model, idata: az.InferenceData, input_variables: list,
                     standardized_observation_data: pd.DataFrame):
    with model:
        thinned_idata = idata.sel(chain=[0], draw=slice(None, None, 1000))

        # Match Observation Length to Original Input Length for Matrix Multiplication
        input_length = len(SCM_SOURCE_DATA)
        data_dict = {}
        observation_length = len(standardized_observation_data)
        number_of_repeats = (input_length + observation_length - 1) // observation_length
        for data_variable in input_variables:
            variable_values = standardized_observation_data[data_variable].values
            repeated_values = np.tile(variable_values, number_of_repeats)[:input_length]
            data_dict[f'{data_variable}_data'] = repeated_values

        pm.set_data(data_dict)

        try:
            predictions = pm.sample_posterior_predictive(thinned_idata, predictions=True, return_inferencedata=False)
        except ValueError as e:
            model.debug(verbose=True)
            raise e

    return predictions


def predictions_to_dataframe(predictions, samples: int = None) -> pd.DataFrame:
    return pd.DataFrame({column.split()[0]: data.flatten()[:samples] for column, data in predictions.items()})


def estimate_posterior_intervention(structural_causal_graph, scm_effect, observation_data, time, friction=0,
                                    desired_speed=0):
    if scm_effect not in ('total', 'direct'):
        raise ValueError("scm_effect must be 'total' or 'direct'")
    elif friction != 0 and desired_speed != 0:
        raise ValueError("Specifying both friction and desired_speed is not currently supported")
    elif time not in ['t0', 't1']:
        raise ValueError("Time must be either 't0' or 't1'")

    file_name = f'scm_{scm_effect}_effect.netcdf'
    file_path = Path().joinpath('structural_causal_models', file_name)
    idata = az.InferenceData.from_netcdf(str(file_path))

    means = SCM_SOURCE_DATA.mean()
    stds = SCM_SOURCE_DATA.std()
    standardized_observation_data = (observation_data[
                                         ['desiredSpeed', 'friction', 'speed', 'waitingTime', 'emergencyBraking',
                                          'collisions']] - means) / stds

    # Generate the SCM model
    model = generate_scm_effect_model(data=SCM_SOURCE_DATA, structural_causal_graph=structural_causal_graph,
                                      effect=scm_effect)
    if scm_effect == 'direct':
        if time == 't1':
            reversed_graph = adjusted_graph.reverse()
            reversed_graph.remove_nodes_from(INDEPENDENT_VARIABLES)
            reversed_model = generate_scm_effect_model(df, reversed_graph, effect='total')
            reversed_idata = az.InferenceData.from_netcdf(str(REVERSED_MODEL_IDATA))
            mediators = sample_posterior(model=reversed_model, idata=reversed_idata, input_variables=OUTCOME_VARIABLES,
                                         standardized_observation_data=standardized_observation_data)
            # bring mediators in same form as observation data
            mediators = predictions_to_dataframe(mediators, samples=len(standardized_observation_data))
            observation_data[mediators.columns] = mediators

        variables = [col for col in standardized_observation_data.columns if col not in OUTCOME_VARIABLES]
    else:
        variables = INTERVENTION_VARIABLES

    predictions = sample_posterior(model=model, idata=idata, input_variables=variables,
                                   standardized_observation_data=standardized_observation_data)

    predictions = predictions_to_dataframe(predictions)
    de_standardized_predictions = pd.DataFrame(predictions)
    for column in de_standardized_predictions.keys():
        de_standardized_predictions[column] = de_standardized_predictions[column].values * stds[column] + means[column]
    de_standardized_predictions = de_standardized_predictions.clip(lower=0.1)

    idata_weibull = {}
    for variable in de_standardized_predictions[['collisions', 'waitingTime']].columns:
        with pm.Model() as weibull_model:
            alpha = pm.TruncatedNormal("alpha", mu=1.5,
                                       sigma=1, lower=0.01)
            beta = pm.TruncatedNormal("beta", mu=de_standardized_predictions[variable].mean(),
                                      sigma=de_standardized_predictions[variable].std(), lower=0.01)
            obs = pm.Weibull(variable, alpha=alpha, beta=beta, observed=de_standardized_predictions[variable])

            idata_weibull[variable] = pm.sample()

    return de_standardized_predictions, idata_weibull

### Interface
How to run an experiment

Preperation: Set Agent, t0 and t1 config as dataclass with 3 attributes: agent (as a pathlib Path), t0 configuration (with values for friction and desired_speed), t1 configuration (with values for friction and desired_speed)

1. load observation for t0 and t1 for given agent and configurations from 1.
2. fit weibull for t0 and t1 on observation data
3. calculate cdf for t0 and t1 on pymc bayesian weibull with parameterizable constraint value
4. Get the difference between the two cdf results

All of this has to be done once for collisions and once for waitingTime

In [ ]:
from numpy import exp
from dataclasses import dataclass
import pandas as pd


@dataclass
class Environment:
    desired_speed: int
    friction: float


@dataclass
class Experiment:
    agent: str
    t0: Environment
    t1: Environment

    def __str__(self):
        return f'{self.agent}_s{self.t1.desired_speed}_f{self.t1.friction}'


@dataclass
class Observation:
    t0: pd.DataFrame
    t1: pd.DataFrame


experiments = [
    Experiment('scratch_s50_f1', Environment(50, 1), Environment(50, 0.5)),
    Experiment('scratch_s50_f1', Environment(50, 1), Environment(80, 1)),
    Experiment('scratch_s80_f0.5', Environment(80, 0.5), Environment(50, 0.5)),
    Experiment('scratch_s80_f0.5', Environment(80, 0.5), Environment(80, 1)),
    Experiment('scratch_s50_f0.5', Environment(50, 0.5), Environment(80, 0.5)),
    Experiment('scratch_s50_f0.5', Environment(50, 0.5), Environment(50, 1)),
]


def generate_prediction_config(experiment: Experiment):
    obs_t0_name = f'{experiment.agent}_shift_s{experiment.t0.desired_speed}_f{float(experiment.t0.friction)}'
    obs_t0_path = Path().joinpath('distribution_shift', 'traces', obs_t0_name, '.summary.csv')
    if not obs_t0_path.exists():
        raise ValueError(f'Could not find observations at {obs_t0_path}, use generate_observations.py to generate')

    obs_t1_name = f'{experiment.agent}_shift_s{experiment.t1.desired_speed}_f{float(experiment.t1.friction)}'
    obs_t1_path = Path().joinpath('distribution_shift', 'traces', obs_t1_name, '.summary.csv')
    if not obs_t1_path.exists():
        raise ValueError(f'Could not find observations at {obs_t0_path}, use generate_observations.py to generate')

    return obs_t0_path, obs_t1_path


def calculate_weibull_cdf_upper(idata: az.InferenceData, value: int):
    shape = az.extract(idata).alpha.mean()
    scale = az.extract(idata).beta.mean()
    cdf_value = weibull_min.cdf(value, c=shape, scale=scale)
    return max(0.01, min(0.99, 1 - cdf_value))


def calculate_cdf_difference(weibull_t0, weibull_t1, value: int) -> float:
    if isinstance(weibull_t0, az.InferenceData):
        cdf_t0 = calculate_weibull_cdf_upper(weibull_t0, value)
        cdf_t1 = calculate_weibull_cdf_upper(weibull_t1, value)
    else:
        cdf_t0 = calculate_weibull_cdf_upper_count(weibull_t0, value)
        cdf_t1 = calculate_weibull_cdf_upper_count(weibull_t1, value)
    if abs(cdf_t0 - cdf_t1) <= 0.01:
        return 0.01 if cdf_t0 > 0 else -0.01
    return cdf_t0 - cdf_t1

def calculate_weibull_cdf_upper_count(data: pd.Series, value: int) -> float:
    violations_count = (data > value).sum()
    return violations_count / len(data)



# for experiment in experiments:
#     observation_files = generate_prediction_config(experiment)
#     weibull_data, _ = fit_observation_distributions(*observation_files)
#     print(calculate_cdf_difference(weibull_data['t0']['collisions'], weibull_data['t1']['collisions'], 20))
#     print(calculate_cdf_difference(weibull_data['t0']['waitingTime'], weibull_data['t1']['waitingTime'], 47))
#
#     print("Done")

interventions = ['total', 'direct']
times = ['t0', 't1']

for experiment in experiments:

    print(experiment)
    obs_t0_path, obs_t1_path = generate_prediction_config(experiment)

    obs_df = {
        't0': pd.read_csv(obs_t0_path)[
            ['desiredSpeed', 'friction', 'speed', 'waitingTime', 'emergencyBraking', 'collisions']],
        't1': pd.read_csv(obs_t1_path)[
            ['desiredSpeed', 'friction', 'speed', 'waitingTime', 'emergencyBraking', 'collisions']]
    }

    base_file_name = f"{experiment.agent}_s{experiment.t1.desired_speed}_f{experiment.t1.friction}"
    for time in times:
        for kind in interventions:
            file_suffix = f"{kind}_{time}"
            file_name = f"{base_file_name}_predictions_{file_suffix}"
            csv_path = Path().joinpath('experiments', 'csv', file_name + ".csv")

            if not csv_path.exists():
                predictions_df, weibull_data = estimate_posterior_intervention(adjusted_graph, kind, obs_df[time], time)
                predictions_df.to_csv(csv_path, index=False)

                for variable, idata in weibull_data.items():
                    netcdf_path = Path().joinpath('experiments', 'netcdf', f'{file_name}_{variable}' + ".netcdf")
                    az.InferenceData.to_netcdf(idata, netcdf_path)

In [ ]:
prediction_values, prediction_weibulls, observation_weibulls = {}, {}, {}
interventions = ['total', 'direct']
times = ['t0', 't1']

for experiment in experiments:

    observation_files = generate_prediction_config(experiment)
    observation_weibulls[str(experiment)], _ = fit_observation_distributions(*observation_files,
                                                                             idata_file_name=str(experiment))

    prediction_values[str(experiment)] = {'t0': {}, 't1': {}}
    prediction_weibulls[str(experiment)] = {'t0': {}, 't1': {}}
    for time in times:
        for kind in interventions:
            base_file_name = f"{str(experiment)}_predictions_{kind}_{time}"
            prediction_values[str(experiment)][time][kind] = pd.read_csv(
                Path().joinpath('experiments', 'csv', base_file_name + '.csv'))[['collisions', 'waitingTime']]

            prediction_weibulls[str(experiment)][time][kind] = {}
            for variable in ['collisions', 'waitingTime']:
                prediction_weibulls[str(experiment)][time][kind][variable] = az.InferenceData.from_netcdf(
                    str(Path().joinpath('experiments', 'netcdf', base_file_name + f'_{variable}.netcdf')))
        current_time_predictions = {
            'total': prediction_values[str(experiment)][time]['total'].copy(),
            'direct': prediction_values[str(experiment)][time]['direct'].copy(),
        }

        #Calculate Weibulls for Indirect Effect
    #     current_time_predictions['total']['collisions'] = sorted(current_time_predictions['total']['collisions'])
    #     current_time_predictions['direct']['collisions'] = sorted(current_time_predictions['direct']['collisions'])
    #     current_time_predictions['total']['waitingTime'] = sorted(current_time_predictions['total']['waitingTime'])
    #     current_time_predictions['direct']['waitingTime'] = sorted(current_time_predictions['direct']['waitingTime'])
    #     current_time_predictions['indirect']['collisions'] = current_time_predictions['total']['collisions'] - \
    #                                                          current_time_predictions['direct']['collisions']
    #     current_time_predictions['indirect']['waitingTime'] = current_time_predictions['total']['waitingTime'] - \
    #                                                           current_time_predictions['direct']['waitingTime']
    #     prediction_values[str(experiment)][time]['indirect'] = {}
    #     prediction_values[str(experiment)][time]['indirect']['collisions'] = current_time_predictions['indirect'][
    #         'collisions']
    #     prediction_values[str(experiment)][time]['indirect']['waitingTime'] = current_time_predictions['indirect'][
    #         'waitingTime']
    #
    # observations_indirect = {
    #     't0': pd.DataFrame(prediction_values[str(experiment)]['t0']['indirect']),
    #     't1': pd.DataFrame(prediction_values[str(experiment)]['t1']['indirect'])
    # }
    # result = fit_observation_distributions(*observation_files, idata_file_name=str(experiment) + '_indirect',
    #                                        observations=observations_indirect)

for experiment in experiments:

    print("Collisions > 20")

    print("Weibull", calculate_cdf_difference(observation_weibulls[str(experiment)]['t0']['collisions'],
                                   observation_weibulls[str(experiment)]['t1']['collisions'], 20))

    print("Observations", calculate_cdf_difference(prediction_values[str(experiment)]['t0']['total']['collisions'],
                                   prediction_values[str(experiment)]['t1']['total']['collisions'], 20))


In [ ]:
import numpy as np


def calculate_explanatory_power(observation, prediction):
    if observation['delta'] * prediction['delta'] > 0:
        return max(0, 1 - abs(observation['delta'] - prediction['delta']) / observation['delta'])
    else:
        return 0


def get_truncated_difference(cdf_t0, cdf_t1):
    if abs(cdf_t0 - cdf_t1) <= 0.01:
        return 0.01 if cdf_t0 > 0 else -0.01
    return cdf_t0 - cdf_t1


def get_explanatory_power_for_value(experiments, variable, value, full_data=False):
    cdfs_observations = {}
    cdfs_predictions_direct = {}
    cdfs_predictions_total = {}
    explanatory_powers = {}

    for experiment in experiments:
        # Observations
        cdfs_observations[str(experiment)] = {}
        cdfs_predictions_direct[str(experiment)] = {}
        cdfs_predictions_total[str(experiment)] = {}

        cdfs_observations[str(experiment)]['t0'] = calculate_weibull_cdf_upper(
            observation_weibulls[str(experiment)]['t0'][variable], value)
        cdfs_observations[str(experiment)]['t1'] = calculate_weibull_cdf_upper(
            observation_weibulls[str(experiment)]['t1'][variable], value)

        cdfs_observations[str(experiment)]['delta'] = get_truncated_difference(cdfs_observations[str(experiment)]['t0'],
                                                                               cdfs_observations[str(experiment)]['t1'])
        # Weibulls
        cdfs_predictions_direct[str(experiment)]['t0'] = calculate_weibull_cdf_upper(
            prediction_weibulls[str(experiment)]['t0']['direct'][variable], value)
        cdfs_predictions_direct[str(experiment)]['t1'] = calculate_weibull_cdf_upper(
            prediction_weibulls[str(experiment)]['t1']['direct'][variable], value)
        cdfs_predictions_direct[str(experiment)]['delta'] = get_truncated_difference(
            cdfs_predictions_direct[str(experiment)]['t0'], cdfs_predictions_direct[str(experiment)]['t1'])

        cdfs_predictions_total[str(experiment)]['t0'] = calculate_weibull_cdf_upper(
            prediction_weibulls[str(experiment)]['t0']['total'][variable], value)
        cdfs_predictions_total[str(experiment)]['t1'] = calculate_weibull_cdf_upper(
            prediction_weibulls[str(experiment)]['t1']['total'][variable], value)
        cdfs_predictions_total[str(experiment)]['delta'] = get_truncated_difference(
            cdfs_predictions_total[str(experiment)]['t0'], cdfs_predictions_total[str(experiment)]['t1'])

        # Direct Values
        # cdfs_predictions_direct[str(experiment)]['t0'] = calculate_weibull_cdf_upper_count(
        #     prediction_values[str(experiment)]['t0']['direct'][variable], value)
        # cdfs_predictions_direct[str(experiment)]['t1'] = calculate_weibull_cdf_upper_count(
        #     prediction_values[str(experiment)]['t1']['direct'][variable], value)
        # cdfs_predictions_direct[str(experiment)]['delta'] = get_truncated_difference(
        #     cdfs_predictions_direct[str(experiment)]['t0'], cdfs_predictions_direct[str(experiment)]['t1'])
        #
        # cdfs_predictions_total[str(experiment)]['t0'] = calculate_weibull_cdf_upper_count(
        #     prediction_values[str(experiment)]['t0']['total'][variable], value)
        # cdfs_predictions_total[str(experiment)]['t1'] = calculate_weibull_cdf_upper_count(
        #     prediction_values[str(experiment)]['t1']['total'][variable], value)
        # cdfs_predictions_total[str(experiment)]['delta'] = get_truncated_difference(
        #     cdfs_predictions_total[str(experiment)]['t0'], cdfs_predictions_total[str(experiment)]['t1'])

        explanatory_powers[str(experiment)] = {
            'direct': calculate_explanatory_power(cdfs_observations[str(experiment)],
                                                  cdfs_predictions_direct[str(experiment)]),
            'total': calculate_explanatory_power(cdfs_observations[str(experiment)],
                                                 cdfs_predictions_total[str(experiment)])
        }

    # cdfs_observations, cdfs_predictions_direct, explanatory_powers
    if full_data:
        return explanatory_powers, cdfs_observations, cdfs_predictions_direct, cdfs_predictions_total
    else:
        return explanatory_powers

explanatory_powers_collisions, cdfs_observations_collisions, cdfs_predictions_direct_collisions, cdfs_predictions_total_collisions = get_explanatory_power_for_value(
    experiments, 'collisions', 39, full_data=True)
explanatory_powers_waitingTime, cdfs_observations_waitingTime, cdfs_predictions_direct_waitingTime, cdfs_predictions_total_waitingTime = get_explanatory_power_for_value(
    experiments, 'waitingTime', 42, full_data=True)


# print('explanatory_powers_collisions = ', explanatory_powers_collisions)
# print('cdfs_observations_collisions = ', cdfs_observations_collisions)
# print('cdfs_predictions_direct_collisions = ', cdfs_predictions_direct_collisions)
# print('cdfs_predictions_total_collisions = ', cdfs_predictions_total_collisions)

rows = []
for exp in explanatory_powers_collisions:
    row = {
        'experiment with c>20 and wT>45': exp,
        'delta_cdf_observation_c': cdfs_observations_collisions[exp]['delta'],
        'delta_cdf_observation_wT': cdfs_observations_waitingTime[exp]['delta'],
        'delta_cdf_prediction_direct_c': cdfs_predictions_direct_collisions[exp]['delta'],
        'delta_cdf_prediction_direct_wT': cdfs_predictions_direct_waitingTime[exp]['delta'],
        'delta_cdf_prediction_total_c': cdfs_predictions_total_collisions[exp]['delta'],
        'delta_cdf_prediction_total_wT': cdfs_predictions_total_waitingTime[exp]['delta'],
        'explanatory_power_direct_c': explanatory_powers_collisions[exp]['direct'],
        'explanatory_power_direct_wT': explanatory_powers_waitingTime[exp]['direct'],
        'explanatory_power_total_c': explanatory_powers_collisions[exp]['total'],
        'explanatory_power_total_wT': explanatory_powers_waitingTime[exp]['total']
    }
    rows.append(row)

# Create the DataFrame with specified column order and set index
columns_order = [
    'delta_cdf_observation_c',
    'delta_cdf_observation_wT',
    'delta_cdf_prediction_direct_c',
    'delta_cdf_prediction_direct_wT',
    'delta_cdf_prediction_total_c',
    'delta_cdf_prediction_total_wT',
    'explanatory_power_direct_c',
    'explanatory_power_direct_wT',
    'explanatory_power_total_c',
    'explanatory_power_total_wT'
]
df = pd.DataFrame(rows).set_index('experiment with c>20 and wT>45')[columns_order].round(4)

# Print the DataFrame
print(df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

experiment_groups = {
    's50_f1': ['scratch_s50_f1_s50_f0.5', 'scratch_s50_f1_s80_f1'],
    's80_f0.5': ['scratch_s80_f0.5_s50_f0.5', 'scratch_s80_f0.5_s80_f1'],
    's50_f0.5': ['scratch_s50_f0.5_s80_f0.5', 'scratch_s50_f0.5_s50_f1'],
}

explanatory_powers_per_value = {}
for i in tqdm(range(100)):
    explanatory_powers_per_value[i] = {
        'collisions': get_explanatory_power_for_value(experiments, 'collisions', i),
        'waitingTime': get_explanatory_power_for_value(experiments, 'waitingTime', i)
    }

rows = []

for value, categories in explanatory_powers_per_value.items():
    collisions = categories['collisions']
    waiting_time = categories['waitingTime']

    # All experiments from collisions keys
    for experiment in collisions:
        row = {
            'value': value,
            'experiment': experiment,
            'collisions_direct': collisions[experiment]['direct'],
            'collisions_total': collisions[experiment]['total'],
            'waitingTime_direct': waiting_time[experiment]['direct'],
            'waitingTime_total': waiting_time[experiment]['total']
        }
        rows.append(row)

df = pd.DataFrame(rows)

for group_name, group_experiments in experiment_groups.items():
    # Filter data for the current group
    group_df = df[df['experiment'].isin(group_experiments)]

    if group_df.empty:
        print(f"No data available for group: {group_name}")
        continue

    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Experiment Group: {group_name}', fontsize=16)
    axes = axes.ravel()  # Flatten the axes array

    # Define columns to plot
    y_columns = ['collisions_direct', 'collisions_total',
                 'waitingTime_direct', 'waitingTime_total']

    # Generate distinct colors for each experiment
    unique_experiments = group_df['experiment'].unique()
    colors = plt.cm.tab10(range(len(unique_experiments)))

    for ax, col in zip(axes, y_columns):
        for idx, exp in enumerate(unique_experiments):
            exp_data = group_df[group_df['experiment'] == exp]

            # Add jitter to x and y values
            jitter_x = exp_data['value'] + np.random.normal(0, 0.005, size=len(exp_data))
            jitter_y = exp_data[col] + np.random.normal(0, 0.001, size=len(exp_data))

            ax.scatter(jitter_x, jitter_y,
                       color=colors[idx], label=exp, alpha=0.7)
        ax.set_xlabel('Constraint > X')
        ax.set_ylabel(f'explanatory power (Pw)  {" ".join(col.split("_"))} effect')
        ax.grid(True, linestyle='--', alpha=0.6)

    # Create a unified legend outside the plots
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right',
               bbox_to_anchor=(1.15, 0.9), title='Experiments')

    plt.tight_layout()
    plt.show()
print("End")